In [1]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image
train_path = 'E:\\Fracture\\train'
test_path = 'E:\\Fracture\\val'
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.load_weights('best_model.h5')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [7]:
from tkinter import filedialog
import tkinter as tk
from PIL import ImageTk, Image
import customtkinter as ctk
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.utils import load_img, img_to_array

class App(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title("Bone Fracture Detection")
        self.geometry(f"{500}x{740}")
        self.head_frame = ctk.CTkFrame(master=self)
        self.head_frame.pack(pady=20, padx=60, fill="both", expand=True)
        self.main_frame = ctk.CTkFrame(master=self)
        self.main_frame.pack(pady=20, padx=60, fill="both", expand=True)
        self.head_label = ctk.CTkLabel(master=self.head_frame, text="Bone Fracture Detection",
                                       font=(ctk.CTkFont("Roboto"), 28))
        self.lis = []
        self.head_label.pack(pady=20, padx=10, anchor="nw", side="left")
        
        self.img_label = ctk.CTkButton(master=self.head_frame, text="Info", command=self.open_image_window,
                                       width=60, height=40)
        self.img_label.pack(pady=10, padx=10, anchor="nw", side="right")

        self.info_label = ctk.CTkLabel(master=self.main_frame,
                                       text="Bone fracture detection system, upload an X-ray image for fracture detection.",
                                       wraplength=300, font=(ctk.CTkFont("Roboto"), 18))
        self.info_label.pack(pady=10, padx=10)
        
        

        
        self.upload_btn = ctk.CTkButton(master=self.main_frame, text="Upload Image", command=self.upload_image)
        self.upload_btn.pack(pady=0, padx=1)
        

        self.frame2 = ctk.CTkFrame(master=self.main_frame, fg_color="transparent", width=256, height=256)
        self.frame2.pack(pady=10, padx=1)
        
        img = Image.open("C:\\Users\\R N Arjun\\OneDrive\\Desktop\\DIP project\\Question_Mark.jpg")
        img_resized = img.resize((int(256 / img.height * img.width), 256))
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2,image=img, text="")
        self.img_label.image = img

        self.img_label = ctk.CTkLabel(master=self.frame2, text="", image=None)
        self.img_label.pack(pady=1, padx=10)

        self.predict_btn = ctk.CTkButton(master=self.main_frame, text="Pre-Process", command=self.preprocess)
        self.predict_btn.pack(pady=0, padx=1)

        self.result_frame = ctk.CTkFrame(master=self.main_frame, fg_color="transparent", width=200, height=100)
        self.result_frame.pack(pady=5, padx=5)

        self.save_btn = ctk.CTkButton(master=self.result_frame, text="Predict", command=self.predict_img)
        self.save_btn.pack(pady=0, padx=1)

        self.loader_label = ctk.CTkLabel(master=self.main_frame, width=100, height=100, text="")
        self.loader_label.pack(pady=3, padx=3)

        self.res1_label = ctk.CTkLabel(master=self.result_frame, text="")
        self.res1_label.pack(pady=5, padx=20)

        self.res2_label = ctk.CTkLabel(master=self.result_frame, text="")
        self.res2_label.pack(pady=5, padx=20)

    def upload_image(self):
        global filename
        filename = filedialog.askopenfilename(filetypes=(("Image files", "*.png;*.jpg;*.jpeg"), ("All files", "*.*")))
        
        self.res1_label.configure(text="")
        self.res2_label.configure(text="")
        self.img_label.configure(image=None)
        img = Image.open(filename)
        self.lis.append(filename)
        img_resized = img.resize((int(256 / img.height * img.width), 256))  # new width & height
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2, image=img, text="")
        self.img_label.image = img
       
    def preprocess(self):
        file_path = self.lis[len(self.lis)-1]
        img = cv2.imread(file_path)
        chThr = [60,60]
        imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        imgBlur = cv2.GaussianBlur(imgGray,(0,0),1)
        ret, thresh1 = cv2.threshold(imgGray, 0, 150, cv2.THRESH_OTSU,cv2.THRESH_BINARY_INV)
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30))
        imgCanny = cv2.Canny(imgBlur,chThr[0],chThr[1]) 
        image = Image.fromarray(imgCanny)
        image.thumbnail((400, 400))
        img_resized = image.resize((int(256 / image.height * image.width ), 256))  # new width & height
        img = ImageTk.PhotoImage(img_resized)
        self.img_label.configure(self.frame2, image=img, text="")
        self.img_label.image = img
        
    def open_image_window(self):
        im = Image.open("C:\\Users\\R N Arjun\\OneDrive\\Desktop\\DIP project\\Rules.png")
        im = im.resize((700, 700))
        im.show()
    def predict_img(self):
        file_path = self.lis[len(self.lis)-1]
        img = cv2.imread(file_path) 
        resized_img = Image.fromarray(img).resize((224, 224))
        resized_array = np.array(resized_img)
        imaga = np.expand_dims(resized_array,axis=0)
        ypred = model.predict(imaga)
        a=ypred[0]
        if a<0.5:
            self.res2_label.configure(text_color="RED", text="Result: Fractured", font=(ctk.CTkFont("Roboto"), 24))
        else:
            self.res2_label.configure(text_color="GREEN", text="Result: Normal", font=(ctk.CTkFont("Roboto"), 24))
      
        
        
        
if __name__ == "__main__":
    app = App()
    app.mainloop()


1/1 [==============================] - 0s 77ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\R N Arjun\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\R N Arjun\anaconda3\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\Users\R N Arjun\AppData\Local\Temp\ipykernel_1268\2607798771.py", line 78, in upload_image
    img = Image.open(filename)
  File "C:\Users\R N Arjun\anaconda3\lib\site-packages\PIL\Image.py", line 3283, in open
    raise UnidentifiedImageError(msg)
PIL.UnidentifiedImageError: cannot identify image file 'C:/Users/R N Arjun/OneDrive/Desktop/DIP project/fracture_datset.zip'


1/1 [==============================] - 0s 60ms/step


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\R N Arjun\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\R N Arjun\anaconda3\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\Users\R N Arjun\AppData\Local\Temp\ipykernel_1268\2607798771.py", line 89, in preprocess
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\R N Arjun\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\R N Arjun\anaconda3\lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\Users\R N Arjun\AppDa

1/1 [==============================] - 0s 86ms/step


KeyboardInterrupt: 